# AOS_DM-49046  AuxTel : update latiss_wep_align

We find CWFS pairs in `AOS_DM-44140_SN_auxTel... `. Here, we just analyse one pair, using the same code as in https://github.com/lsst-ts/ts_externalscripts/blob/develop/python/lsst/ts/externalscripts/auxtel/latiss_wep_align.py : 

In [2]:
# Get the defocal exposure
dataIdExtra = {'instrument': 'LATISS', 
          'exposure': 2024040300257, 
           'detector':0}

dataIdIntra = {'instrument': 'LATISS', 
          'exposure': 2024040300258, 
           'detector':0}

In [3]:
from lsst.summit.utils.bestEffort import BestEffortIsr
bestEffort = BestEffortIsr(embargo=False)
exposure_intra = bestEffort.getExposure(dataIdIntra)
exposure_extra = bestEffort.getExposure(dataIdExtra)

INFO:lsst.summit.utils.bestEffort:Instantiating butler with collections=['LATISS/raw/all', 'LATISS/calib', 'LATISS/runs/quickLook']
INFO:lsst.summit.utils.bestEffort:Found a ready-made quickLookExp in the repo. Returning that.
INFO:lsst.summit.utils.bestEffort:Found a ready-made quickLookExp in the repo. Returning that.


In [4]:
from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask
quick_frame_measurement_config = QuickFrameMeasurementTask.ConfigClass()
quick_frame_measurement_task = QuickFrameMeasurementTask(
    config=quick_frame_measurement_config
)


In [5]:
import numpy as np 
def side(_side = 192 * 1.1, dz=0.8) -> int:
    # must be an even number
    return int(np.ceil(_side * dz / 1.5 / 2.0) * 2)

In [6]:
donut_diameter=2*side()

In [7]:

result_intra = quick_frame_measurement_task.run(
    exposure_intra, donutDiameter=donut_diameter
)
result_extra = quick_frame_measurement_task.run(
    exposure_extra, donutDiameter=donut_diameter
)

INFO:lsst.quickFrameMeasurementTask:Found 6 sources in exposure
INFO:lsst.quickFrameMeasurementTask:Measured 6 of 6 sources in exposure
INFO:lsst.quickFrameMeasurementTask:Found 13 sources in exposure
INFO:lsst.quickFrameMeasurementTask:Measured 13 of 13 sources in exposure



Run parts of `latiss_wep_align`,  ensure that the output is compatible  with `cutOutDonuts` : 

In [14]:
from astropy.table import Table, QTable
import astropy.units as u
from lsst.ts.wep.task.generateDonutCatalogUtils import addVisitInfoToCatTable

def get_donut_catalog(result, exposure ):#:wcs):# -> astropy.table.QTable:
    """Get the donut catalog, used by wep, from the quick frame measurement
    result.

    Parameters
    ----------
    result : `Struct`
        Result of `QuickFrameMeasurementTask`.
    wcs : `SkyWcs`
        Exposure WCS, to compute Ra/Dec.

    Returns
    -------
    donut_catalog : `pandas.DataFrame`
        Donut catalog.
    """
    wcs = exposure.getWcs()
    ra, dec = wcs.pixelToSkyArray(
        result.brightestObjCentroidCofM[0],
        result.brightestObjCentroidCofM[1],
        degrees=False,
    )
    donutTable = QTable()
    donutTable["coord_ra"] = ra * u.rad
    donutTable["coord_dec"] = dec * u.rad
    donutTable["centroid_x"] = [result.brightestObjCentroidCofM[0]] * u.pixel
    donutTable["centroid_y"] = [result.brightestObjCentroidCofM[1]] * u.pixel
    donutTable["source_flux"] = [result.brightestObjApFlux70] * u.nJy
    donutTable.meta["blend_centroid_x"] = ""
    donutTable.meta["blend_centroid_y"] = ""
    donutTable.sort("source_flux", reverse=True)
    donut_catalog = addVisitInfoToCatTable(exposure, donutTable)
    return donutTable

In [15]:
donut_catalog_intra = get_donut_catalog(result_intra, exposure_intra)
donut_catalog_extra = get_donut_catalog(result_extra, exposure_extra)


That way donut catalog has metadata:

In [16]:
donut_catalog_intra.meta["visit_info"]

{'boresight_ra': <Quantity 111.21219281 deg>,
 'boresight_dec': <Quantity -19.01229339 deg>,
 'boresight_alt': <Quantity 78.45859477 deg>,
 'boresight_az': <Quantity 344.89117144 deg>,
 'boresight_rot_angle': <Quantity 90.05036013 deg>,
 'rot_type_name': 'SKY',
 'rot_type_value': 1,
 'boresight_par_angle': <Quantity 166.15592879 deg>,
 'focus_z': <Quantity -0.80109996 mm>,
 'mjd': 60403.98009460076,
 'visit_id': 2024040300258,
 'instrument_label': 'LATISS',
 'observatory_elevation': <Quantity 2663.0051116 m>,
 'observatory_latitude': <Quantity -30.24478892 deg>,
 'observatory_longitude': <Quantity -70.74768621 deg>,
 'ERA': <Quantity 185.15247663 deg>,
 'exposure_time': <Quantity 30. s>}

In [17]:
from lsst.ts.wep.task.calcZernikesTask import (
    CalcZernikesTask,
    CalcZernikesTaskConfig,
)
from lsst.ts.wep.task.cutOutDonutsScienceSensorTask import (
    CutOutDonutsScienceSensorTask,
    CutOutDonutsScienceSensorTaskConfig,
)

In [18]:
cut_out_config = CutOutDonutsScienceSensorTaskConfig()
cut_out_config.donutStampSize = donut_diameter
cut_out_config.opticalModel = "onAxis"
cut_out_config.initialCutoutPadding = 40
cut_out_task = CutOutDonutsScienceSensorTask(config=cut_out_config)


In [19]:
donut_catalog_extra

coord_ra,coord_dec,centroid_x,centroid_y,source_flux
rad,rad,pix,pix,nJy
float64,float64,float64,float64,float64
1.9410281820183282,-0.33183933400997156,2118.2924057571972,2026.0640304914314,165058733.70458984


In [20]:
from lsst.obs.lsst import Latiss
camera = Latiss.getCamera()

cut_out_output = cut_out_task.run(
    [exposure_extra, exposure_intra],
    [donut_catalog_extra, donut_catalog_intra],
    camera,
)

INFO:lsst.CutOutDonutsScienceSensorTask:Median Recentering Shift: (0.0, 2.0)
/sdf/data/rubin/user/scichris/WORK/aos_packages/ts_wep/python/lsst/ts/wep/task/cutOutDonutsBase.py:369: RuntimeWarning: Mean of empty slice.
  signalMean = imageArray[donutMask].mean()  # per pixel
/sdf/group/rubin/sw/conda/envs/lsst-scipipe-9.0.0/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/sdf/data/rubin/user/scichris/WORK/aos_packages/ts_wep/python/lsst/ts/wep/task/cutOutDonutsBase.py:419: RuntimeWarning: invalid value encountered in scalar divide
  sn = ttlSignalSum / ttlNoiseBkgndVariance
INFO:lsst.CutOutDonutsScienceSensorTask:Median Recentering Shift: (1.0, -2.0)
/sdf/data/rubin/user/scichris/WORK/aos_packages/ts_wep/python/lsst/ts/wep/task/cutOutDonutsBase.py:369: RuntimeWarning: Mean of empty slice.
  signalMean = imageArray[donutMask].mean()  # per pixel
/sdf/group/rubin/sw/conda/envs/lsst-scipipe-9.

In [21]:
config = CalcZernikesTaskConfig()
config.doDonutStampSelector = False
task = CalcZernikesTask(config=config, name="Base Task")

task_output = task.run(
    cut_out_output.donutStampsExtra, cut_out_output.donutStampsIntra
)

INFO:CombineZernikesSigmaClipTask:MaxZernClip config: 3. MaxZernClip used: 3.
INFO:CombineZernikesSigmaClipTask:Using 1 pairs out of 1 in final Zernike estimate.


In [22]:
task_output

Struct(outputZernikesAvg=[[ 0.05152419  0.05700915 -0.16035153 -0.00609559  0.0222052  -0.06402742
  -0.00109236  0.00504777  0.0085272   0.00291786  0.02218127 -0.015191
   0.00267969 -0.00093012 -0.00057789 -0.00398313  0.00635297 -0.01819257
   0.00978391 -0.00179852 -0.00287736  0.00496533 -0.0028662  -0.01952071
  -0.01028967]]; outputZernikesRaw=[[ 0.05152419  0.05700915 -0.16035153 -0.00609559  0.0222052  -0.06402742
  -0.00109236  0.00504777  0.0085272   0.00291786  0.02218127 -0.015191
   0.00267969 -0.00093012 -0.00057789 -0.00398313  0.00635297 -0.01819257
   0.00978391 -0.00179852 -0.00287736  0.00496533 -0.0028662  -0.01952071
  -0.01028967]]; zernikes= label  used ...         Z27                 Z28        
             ...          nm                  nm        
------- ---- ... ------------------- -------------------
average True ... -19.520708084106445 -10.289671897888184
  pair1 True ... -19.520708084106445 -10.289671897888184; donutQualityTable=<No columns>)